# ORM and RELATIONSHIPS (Done right)
*Author: Marco Prenassi*   
*License: Creative Commons Attribution 4.0 International*    

In [1]:
!pip install sqlalchemy

In [41]:
# Just to be sure, let's delete the database and..
!rm ./databases/orm_database.db

In [42]:
# Create a new sqlite database
from sqlalchemy import create_engine
engine = create_engine("sqlite:///databases/orm_database.db", echo=True)


In [43]:
from sqlalchemy.exc import IntegrityError # Just to catch every double insertion

In [44]:
# Let's import some modules
from sqlalchemy import String
from sqlalchemy.orm import DeclarativeBase
from sqlalchemy.orm import Mapped
from sqlalchemy.orm import mapped_column
# These are new modules to manage relationships
from sqlalchemy.orm import relationship
from typing import List
from typing import Optional
from sqlalchemy import ForeignKey

In [45]:
# Let's create our engine
from sqlalchemy import create_engine
engine = create_engine("sqlite:///databases/orm_database.db", echo=True)
# Create all the table, using the definition in Base (that's why we used it)


In [46]:
# let's drop all to be sure
try:
    Base.metadata.drop_all(engine)
except:
    pass # LIKE ALWAYS, NEVER DO THIS. 

2024-10-14 06:19:39,446 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-10-14 06:19:39,501 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("association_table")
2024-10-14 06:19:39,505 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-10-14 06:19:39,514 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("association_table")
2024-10-14 06:19:39,514 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-10-14 06:19:39,515 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("left_table")
2024-10-14 06:19:39,515 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-10-14 06:19:39,516 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("left_table")
2024-10-14 06:19:39,517 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-10-14 06:19:39,517 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("right_table")
2024-10-14 06:19:39,517 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-10-14 06:19:39,518 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("right_table")
2024-10-14 06:19:39,518 INFO sql

#### ONE TO ONE Relationship

In [47]:
class Base(DeclarativeBase):
    pass

class Parent(Base):
    __tablename__ = "parent_table"
    name: Mapped[str] = mapped_column(String[50],primary_key=True)
    child: Mapped["Child"] = relationship(back_populates="parent")

class Child(Base):
    __tablename__ = "child_table"

    name: Mapped[str] = mapped_column(String[50],primary_key=True)
    parent_id: Mapped[str] = mapped_column(ForeignKey("parent_table.name"))
    parent: Mapped["Parent"] = relationship(back_populates="child")

Base.metadata.create_all(engine)

2024-10-14 06:19:40,625 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-10-14 06:19:40,626 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("parent_table")
2024-10-14 06:19:40,627 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-10-14 06:19:40,627 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("parent_table")
2024-10-14 06:19:40,628 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-10-14 06:19:40,628 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("child_table")
2024-10-14 06:19:40,629 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-10-14 06:19:40,629 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("child_table")
2024-10-14 06:19:40,630 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-10-14 06:19:40,632 INFO sqlalchemy.engine.Engine 
CREATE TABLE parent_table (
	name VARCHAR NOT NULL, 
	PRIMARY KEY (name)
)


2024-10-14 06:19:40,633 INFO sqlalchemy.engine.Engine [no key 0.00032s] ()
2024-10-14 06:19:40,636 INFO sqlalchemy.engine.Engine 
CREATE TABLE child_table (
	n

In [48]:
for row in Base.registry.mappers:
    print(row)

from sqlalchemy.orm import Session
with Session(engine) as session:
    try:
        u = Child(name = 'John')
        p = Parent(name = 'Carl')
        p.child = u
        session.add(u)
        session.commit()
    except IntegrityError as e:
        print(e)
    

Mapper[Child(child_table)]
Mapper[Parent(parent_table)]
2024-10-14 06:19:41,130 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-10-14 06:19:41,134 INFO sqlalchemy.engine.Engine INSERT INTO parent_table (name) VALUES (?)
2024-10-14 06:19:41,135 INFO sqlalchemy.engine.Engine [generated in 0.00210s] ('Carl',)
2024-10-14 06:19:41,138 INFO sqlalchemy.engine.Engine INSERT INTO child_table (name, parent_id) VALUES (?, ?)
2024-10-14 06:19:41,138 INFO sqlalchemy.engine.Engine [generated in 0.00037s] ('John', 'Carl')
2024-10-14 06:19:41,139 INFO sqlalchemy.engine.Engine COMMIT


In [49]:
from sqlalchemy import select
with Session(engine) as session:
    stmt = select(Parent)
    for parent in session.scalars(stmt):
        print(f"child: {parent.child.name}")

2024-10-14 06:19:41,909 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-10-14 06:19:41,924 INFO sqlalchemy.engine.Engine SELECT parent_table.name 
FROM parent_table
2024-10-14 06:19:41,928 INFO sqlalchemy.engine.Engine [generated in 0.00481s] ()
2024-10-14 06:19:41,937 INFO sqlalchemy.engine.Engine SELECT child_table.name AS child_table_name, child_table.parent_id AS child_table_parent_id 
FROM child_table 
WHERE ? = child_table.parent_id
2024-10-14 06:19:41,938 INFO sqlalchemy.engine.Engine [generated in 0.00120s] ('Carl',)
child: John
2024-10-14 06:19:41,940 INFO sqlalchemy.engine.Engine ROLLBACK


#### One To Many
*A one to many relationship places a foreign key on the child table referencing the parent. relationship() is then specified on the parent, as referencing a collection of items represented by the child:*

In [50]:
# let's drop all to be sure
try:
    Base.metadata.drop_all(engine)
except:
    pass # LIKE ALWAYS, NEVER DO THIS. 

2024-10-14 06:19:45,451 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-10-14 06:19:45,454 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("parent_table")
2024-10-14 06:19:45,454 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-10-14 06:19:45,459 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("child_table")
2024-10-14 06:19:45,460 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-10-14 06:19:45,462 INFO sqlalchemy.engine.Engine 
DROP TABLE child_table
2024-10-14 06:19:45,463 INFO sqlalchemy.engine.Engine [no key 0.00048s] ()
2024-10-14 06:19:45,468 INFO sqlalchemy.engine.Engine 
DROP TABLE parent_table
2024-10-14 06:19:45,468 INFO sqlalchemy.engine.Engine [no key 0.00045s] ()
2024-10-14 06:19:45,470 INFO sqlalchemy.engine.Engine COMMIT


In [51]:
class Base(DeclarativeBase):
    pass

class Parent(Base):
    __tablename__ = "parent_table"

    name: Mapped[str] = mapped_column(String[50], primary_key=True)
    children: Mapped[List["Child"]] = relationship(back_populates="parent")
    __table_args__ = {'extend_existing' : True}

class Child(Base):
    __tablename__ = "child_table"

    name: Mapped[str] = mapped_column(String[50], primary_key=True)
    parent_id: Mapped[int] = mapped_column(ForeignKey("parent_table.name"))
    parent: Mapped["Parent"] = relationship(back_populates="children")
    __table_args__ = {'extend_existing' : True}

Base.metadata.create_all(engine)



2024-10-14 06:19:46,272 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-10-14 06:19:46,273 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("parent_table")
2024-10-14 06:19:46,273 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-10-14 06:19:46,274 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("parent_table")
2024-10-14 06:19:46,274 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-10-14 06:19:46,275 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("child_table")
2024-10-14 06:19:46,275 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-10-14 06:19:46,275 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("child_table")
2024-10-14 06:19:46,276 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-10-14 06:19:46,277 INFO sqlalchemy.engine.Engine 
CREATE TABLE parent_table (
	name VARCHAR NOT NULL, 
	PRIMARY KEY (name)
)


2024-10-14 06:19:46,278 INFO sqlalchemy.engine.Engine [no key 0.00047s] ()
2024-10-14 06:19:46,281 INFO sqlalchemy.engine.Engine 
CREATE TABLE child_table (
	n

In [52]:
# Let's create all the data:
for row in Base.registry.mappers:
    print(row)

from sqlalchemy.orm import Session
with Session(engine) as session:
    try:
        u = [Child(name = 'Jacob'), Child(name = 'Igor'), Child(name = 'Veronica')]
        p = Parent(name = 'Mister')
        p.children.extend(u)
        session.add_all(u)
        session.commit()
    except IntegrityError as e:
        print(e)

Mapper[Child(child_table)]
Mapper[Parent(parent_table)]
2024-10-14 06:19:46,991 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-10-14 06:19:46,994 INFO sqlalchemy.engine.Engine INSERT INTO parent_table (name) VALUES (?)
2024-10-14 06:19:46,994 INFO sqlalchemy.engine.Engine [generated in 0.00077s] ('Mister',)
2024-10-14 06:19:46,997 INFO sqlalchemy.engine.Engine INSERT INTO child_table (name, parent_id) VALUES (?, ?)
2024-10-14 06:19:46,999 INFO sqlalchemy.engine.Engine [generated in 0.00171s] [('Jacob', 'Mister'), ('Igor', 'Mister'), ('Veronica', 'Mister')]
2024-10-14 06:19:47,000 INFO sqlalchemy.engine.Engine COMMIT


In [53]:
from sqlalchemy import select
engine.echo = False
with Session(engine) as session:
    stmt = select(Parent)
    for parent in session.scalars(stmt):
        print(f"parent: {parent.name}")
        for child in parent.children:
            print(f"  child: {child.name}")

parent: Mister
  child: Jacob
  child: Igor
  child: Veronica


In [54]:
try:
    Base.metadata.drop_all(engine)
except:
    pass # LIKE ALWAYS, NEVER DO THIS. 

#### Many To Many
*This is a really useful relationship, but the complexity grows, we need to extend our database*    
*creating a new cross table*

In [55]:
from __future__ import annotations

from sqlalchemy import Column
from sqlalchemy import Table
from sqlalchemy import ForeignKey
from sqlalchemy import Integer
from sqlalchemy.orm import Mapped
from sqlalchemy.orm import mapped_column
from sqlalchemy.orm import DeclarativeBase
from sqlalchemy.orm import relationship


class Base(DeclarativeBase):
    pass


association_table = Table(
    "association_table",
    Base.metadata,
    Column("left_id", ForeignKey("left_table.id"), primary_key=True),
    Column("right_id", ForeignKey("right_table.id"), primary_key=True),
)


class Parent(Base):
    __tablename__ = "left_table"

    id: Mapped[int] = mapped_column(primary_key=True)
    children: Mapped[List[Child]] = relationship(
        secondary=association_table, back_populates="parents"
    )


class Child(Base):
    __tablename__ = "right_table"

    id: Mapped[int] = mapped_column(primary_key=True)
    parents: Mapped[List[Parent]] = relationship(
        secondary=association_table, back_populates="children"
    )
Base.metadata.create_all(engine)

In [58]:
from sqlalchemy import insert


with Session(engine) as session:
    try:
        p1 = Parent(id = 2)
        u1 = Child(id = 2)
        p1.children.append(u1)
        session.add(p1)
        session.add(u1)
        session.commit()
    except IntegrityError as e:
        print(e)
    